In  a docker terminal run
jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root

then open jupyter in the printed link (such as http://127.0.0.1:8888/tree?token=4d9d578397aede50fc5bd2d92794b562a6bbcbc73b2dfe51)

CODE HERE, REFRESH AND EXECUTE IN THE BROWSER

In [3]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append('/app')

os.environ['DJANGO_SETTINGS_MODULE'] = 'app.settings'
os.environ['PYTHONPATH'] = '/app'

django.setup()


In [30]:
from django.conf import settings
import requests
from requests.auth import HTTPBasicAuth
import json

In [52]:
GEOSERVER_URL = settings.GEOSERVER_URL + '/rest'
GEOSERVER_USER = settings.GEOSERVER_USER
GEOSERVER_PASS = settings.GEOSERVER_PASS
WORKSPACE = "spreewassern_raster"

In [7]:


def publish_raster_geoserver(tif_path, layer_name, style_name="style_raster_percent"):
    """
    Publishes a GeoTIFF to GeoServer as a coverage store and attaches an existing style.
    """
    store_name = layer_name
    headers = {"Content-type": "image/tiff"}

    # 1. Upload GeoTIFF to a new coverage store (overwrite if exists)
    url = f"{GEOSERVER_URL}/workspaces/{WORKSPACE}/coveragestores/{store_name}/file.tif"
    with open(tif_path, "rb") as f:
        r = requests.put(
            url,
            data=f,
            headers=headers,
            auth=HTTPBasicAuth(GEOSERVER_USER, GEOSERVER_PASS),
            params={"configure": "all", "coverageName": layer_name}  # ensures layer name matches
        )
    r.raise_for_status()

    # 2. Attach existing style
    layer_url = f"{GEOSERVER_URL}/layers/{WORKSPACE}:{layer_name}"
    r = requests.put(
        layer_url,
        headers={"Content-type": "application/xml"},
        data=f"<layer><defaultStyle><name>{style_name}</name></defaultStyle></layer>",
        auth=HTTPBasicAuth(GEOSERVER_USER, GEOSERVER_PASS)
    )
    r.raise_for_status()


In [53]:
GEOSERVER_URL

'http://geoserver:8080/geoserver/rest'

In [80]:

def create_store(tif_path, layer_name, style_name="style_raster_percent"):
    """
    Publishes a GeoTIFF to GeoServer as a coverage store and attaches an existing style.
    """
    
    store_name = layer_name

    # create a new store in the workspace
    payload = {
        "coverageStore": {
            "name": layer_name,
            "type": "GeoTIFF",
            "enabled": True,
            "workspace": WORKSPACE,
            "url": f"file:{tif_path}"
        }
    }

    # 1. Upload GeoTIFF to a new coverage store (overwrite if exists)
    url = f"{GEOSERVER_URL}/workspaces/{WORKSPACE}/coveragestores"

    r = requests.post(
        url,
        auth=HTTPBasicAuth(GEOSERVER_USER, GEOSERVER_PASS),
        headers={"Content-type": "application/json"},
        data=json.dumps(payload)
    )

    r.raise_for_status()

# TODO PUT/workspaces/{workspace}/coveragestores/{store}/reset
# Reset the caches related to this specific coverage store.

In [82]:
create_store('/opt/geoserver_data/raster_data/4_mar_result.tif', '4_mar_result')


In [78]:
# post the layer
# /workspaces/{workspaceName}/coveragestores/{coveragestoreName}/coverages

def publish_layer(layer_name, style_name="style_raster_percent"):
    """
    Publishes a GeoTIFF to GeoServer as a coverage store and attaches an existing style.
    """
    coverage_url = f"{GEOSERVER_URL}/workspaces/{WORKSPACE}/coveragestores/{layer_name}/coverages"
    
    xml_payload = f"""
        <coverage>
            <name>{layer_name}</name>
            <nativeCoverageName>{layer_name}</nativeCoverageName>
            <title>{layer_name}</title>
            <srs>EPSG:25833</srs>
            <nativeFormat>GeoTIFF</nativeFormat>
            <enabled>true</enabled>
        </coverage>
        """
    r = requests.post(
        coverage_url,
        auth=HTTPBasicAuth(GEOSERVER_USER, GEOSERVER_PASS),
        headers={"Content-Type": "text/xml"},
        data=xml_payload
    )
    r.raise_for_status()

In [79]:
publish_layer('2_mar_result')

HTTPError: 500 Server Error:  for url: http://geoserver:8080/geoserver/rest/workspaces/spreewassern_raster/coveragestores/2_mar_result/coverages

In [95]:
GEOSERVER_URL

'http://geoserver:8080/geoserver/rest'

In [96]:

def x_publish_raster_geoserver(layer_name, style_name="style_raster_percent"):
    """
    Publishes a GeoTIFF to GeoServer as a coverage store and attaches an existing style.
    """
    cache_url = f"http://geoserver:8080/geoserver/gwc/rest/seed/{WORKSPACE}:{layer_name}.xml"
    r = requests.delete(
        cache_url,
        auth=HTTPBasicAuth(GEOSERVER_USER, GEOSERVER_PASS)
    )
    print(f"Cache for {layer_name} cleared (status {r.status_code})")

    url = f"{GEOSERVER_URL}/workspaces/{WORKSPACE}/coveragestores/{layer_name}/file.geotiff"
    with open(f"/app/raster_data/{layer_name}.tif", "rb") as f:
        r = requests.put(
            url,
            auth=HTTPBasicAuth(GEOSERVER_USER, GEOSERVER_PASS),
            headers={"Content-type": "image/tiff"},
            params={"configure": "all", "coverageName": layer_name},
            data=f
        )
    r.raise_for_status()
    print(f"Raster {layer_name} uploaded successfully")

    # Apply style
    layer_url = f"{GEOSERVER_URL}/layers/{WORKSPACE}:{layer_name}"
    style_xml = f"""
    <layer>
        <defaultStyle>
            <name>{style_name}</name>
        </defaultStyle>
    </layer>
    """
    r = requests.put(
        layer_url,
        auth=HTTPBasicAuth(GEOSERVER_USER, GEOSERVER_PASS),
        headers={"Content-type": "application/xml"},
        data=style_xml
    )
    r.raise_for_status()
    print(f"Style '{style_name}' applied to layer '{layer_name}'")


In [97]:
x_publish_raster_geoserver('9_mar_result')

Cache for 9_mar_result cleared (status 405)
Raster 9_mar_result uploaded successfully
Style 'style_raster_percent' applied to layer '9_mar_result'


In [ ]:
payload = {
    "coverage": {
        "abstract": "Digital elevation model for the Spearfish region.\r\n\r\nsfdem is a Tagged Image File Format with Geographic information",
        "defaultInterpolationMethod": "nearest neighbor",
        "description": "Generated from sfdem",
        "dimensions": {
          "coverageDimension": [
            {
              "description": "GridSampleDimension[-9.999999933815813E36,-9.999999933815813E36]",
              "name": "GRAY_INDEX",
              "range": {
                "max": -9.999999933815813e+36,
                "min": -9.999999933815813e+36
              }
            }
          ]
        },
        "enabled": true,
        "grid": {
          "@dimension": "2",
          "crs": "EPSG:26713",
          "range": {
            "high": "634 477",
            "low": "0 0"
          },
          "transform": {
            "scaleX": 30,
            "scaleY": -30,
            "shearX": 0,
            "shearY": 0,
            "translateX": 589995,
            "translateY": 4927995
          }
        },
        "interpolationMethods": {
          "string": [
            "nearest neighbor",
            "bilinear",
            "bicubic"
          ]
        },
        "keywords": {
          "string": [
            "WCS",
            "sfdem",
            "sfdem",
            "type\\@language=fr\\;\\@vocabulary=test\\;"
          ]
        },
        "latLonBoundingBox": {
          "crs": "EPSG:4326",
          "maxx": -103.62940739432703,
          "maxy": 44.5016011535299,
          "minx": -103.87108701853181,
          "miny": 44.370187074132616
        },
        "metadata": {
          "entry": [
            {
              "@key": "elevation",
              "dimensionInfo": {
                "enabled": false
              }
            },
            {
              "$": "10",
              "@key": "cacheAgeMax"
            },
            {
              "@key": "time",
              "dimensionInfo": {
                "defaultValue": "",
                "enabled": false
              }
            },
            {
              "$": "true",
              "@key": "cachingEnabled"
            },
            {
              "$": "sfdem_sfdem",
              "@key": "dirName"
            }
          ]
        },
        "name": "sfdem",
        "namespace": {
          "href": "http://localhost:8075/geoserver/restng/namespaces/sf.json",
          "name": "sf"
        },
        "nativeBoundingBox": {
          "crs": {
            "$": "EPSG:26713",
            "@class": "projected"
          },
          "maxx": 609000,
          "maxy": 4928010,
          "minx": 589980,
          "miny": 4913700
        },
        "nativeCRS": {
          "$": "PROJCS[\"NAD27 / UTM zone 13N\", \n  GEOGCS[\"NAD27\", \n    DATUM[\"North American Datum 1927\", \n      SPHEROID[\"Clarke 1866\", 6378206.4, 294.9786982138982, AUTHORITY[\"EPSG\",\"7008\"]], \n      TOWGS84[2.478, 149.752, 197.726, 0.526, -0.498, 0.501, 0.685], \n      AUTHORITY[\"EPSG\",\"6267\"]], \n    PRIMEM[\"Greenwich\", 0.0, AUTHORITY[\"EPSG\",\"8901\"]], \n    UNIT[\"degree\", 0.017453292519943295], \n    AXIS[\"Geodetic longitude\", EAST], \n    AXIS[\"Geodetic latitude\", NORTH], \n    AUTHORITY[\"EPSG\",\"4267\"]], \n  PROJECTION[\"Transverse_Mercator\", AUTHORITY[\"EPSG\",\"9807\"]], \n  PARAMETER[\"central_meridian\", -105.0], \n  PARAMETER[\"latitude_of_origin\", 0.0], \n  PARAMETER[\"scale_factor\", 0.9996], \n  PARAMETER[\"false_easting\", 500000.0], \n  PARAMETER[\"false_northing\", 0.0], \n  UNIT[\"m\", 1.0], \n  AXIS[\"Easting\", EAST], \n  AXIS[\"Northing\", NORTH], \n  AUTHORITY[\"EPSG\",\"26713\"]]",
          "@class": "projected"
        },
        "nativeFormat": "GeoTIFF",
        "nativeName": "sfdem",
        "requestSRS": {
          "string": [
            "EPSG:26713"
          ]
        },
        "responseSRS": {
          "string": [
            "EPSG:26713"
          ]
        },
        "srs": "EPSG:26713",
        "store": {
          "@class": "coverageStore",
          "href": "http://localhost:8075/geoserver/restng/workspaces/sf/coveragestores/sfdem.json",
          "name": "sf:sfdem"
        },
        "supportedFormats": {
          "string": [
            "ARCGRID",
            "IMAGEMOSAIC",
            "GTOPO30",
            "GEOTIFF",
            "GIF",
            "PNG",
            "JPEG",
            "TIFF"
          ]
        },
        "title": "Spearfish elevation"
      }
    }

<Response [200]>

In [76]:
def publish_layer(layer_name, style_name="style_raster_percent"):
    """
    Publishes the coverage (layer) from an existing coverage store and applies an existing style.
    """
    # 1. Publish coverage (layer)
    # http://localhost:8080/geoserver/rest/workspaces/spreewassern_raster/coveragestores/2_mar_result/coverages
    coverage_url = f"{GEOSERVER_URL}/workspaces/{WORKSPACE}/coveragestores/{layer_name}/coverages"
    print(coverage_url)
    xml_payload = f"""
        <coverage>
            <name>{layer_name}</name>
            <nativeCoverageName>{layer_name}</nativeCoverageName>
            <title>{layer_name}</title>
            <srs>EPSG:25833</srs>
            <nativeFormat>GeoTIFF</nativeFormat>
            <enabled>true</enabled>
        </coverage>
        """
    
    r = requests.post(
        coverage_url,
        auth=HTTPBasicAuth(GEOSERVER_USER, GEOSERVER_PASS),
        headers={"Content-Type": "text/xml"},
        data=xml_payload
    )
   # r.raise_for_status()


In [77]:
publish_layer('2_mar_result')

http://geoserver:8080/geoserver/rest/workspaces/spreewassern_raster/coveragestores/2_mar_result/coverages


In [109]:
def delete_geoserver_layer(workspace, layer_name):
    """
    Clears GeoWebCache tiles for a specific layer in GeoServer.
    """
    url = f"{GEOSERVER_URL}/gwc/rest/layers/{workspace}:{layer_name}.xml"
    
    r = requests.delete(
        url,
        auth=HTTPBasicAuth(GEOSERVER_USER, GEOSERVER_PASS),
    )

    if r.status_code in (200, 202, 204):
        print(f"✅ Cache for {workspace}:{layer_name} cleared successfully")
    else:
        print(f"⚠️ Failed to clear cache: {r.status_code} - {r.text}")
        r.raise_for_status()


In [111]:
def clear_gwc_cache(workspace, layer_name):
    url = f"{settings.GEOSERVER_URL}/gwc/rest/seed/{workspace}:{layer_name}.xml"
    r = requests.delete(
        url,
        auth=HTTPBasicAuth(settings.GEOSERVER_USER, settings.GEOSERVER_PASS),
        headers={"Accept": "application/xml"}
    )
    if r.status_code in (200, 202, 204):
        print(f"✅ Cache for {workspace}:{layer_name} cleared successfully")
    elif r.status_code == 404:
        print(f"ℹ️ No cache existed yet for {workspace}:{layer_name}")
    else:
        print(f"⚠️ Failed to clear cache: {r.status_code} - {r.text}")
        r.raise_for_status()


In [112]:
clear_gwc_cache("spreewassern_raster", "9_mar_result")

⚠️ Failed to clear cache: 405 - 


HTTPError: 405 Client Error:  for url: http://geoserver:8080/geoserver/gwc/rest/seed/spreewassern_raster:9_mar_result.xml

In [110]:
delete_geoserver_layer("spreewassern_raster", "9_mar_result")

⚠️ Failed to clear cache: 404 - Unknown layer: spreewassern_raster:9_mar_result


HTTPError: 404 Client Error:  for url: http://geoserver:8080/geoserver/gwc/rest/layers/spreewassern_raster:9_mar_result.xml